# Imports

In [37]:
import math
import pandas as pd
import pennylane as qml

from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [38]:
np.random.seed(42)
initial_params = np.random.random([14])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 200
TEST_SIZE = 0.2
STEP_SIZE = 0.01

# Fetch Dataset

In [39]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

# Import dataset

In [40]:
examples = mnist.data
classes = mnist.target

x = []
y = []
for (example, label) in zip(examples, classes):
    if label == "0":
        x.append(example)
        y.append(-1)
    elif label == "1":
        x.append(example)
        y.append(1)

In [41]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [42]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

# Circuit creation

In [43]:
device = qml.device("default.qubit", wires=8)

In [44]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(params[2], wires=2)
    qml.RY(params[3], wires=3)
    qml.CNOT(wires=[2, 3])
    qml.RY(params[4], wires=4)
    qml.RY(params[5], wires=5)
    qml.CNOT(wires=[5, 4])
    qml.RY(params[6], wires=6)
    qml.RY(params[7], wires=7)
    qml.CNOT(wires=[7, 6])

    # Second layer
    qml.RY(params[8], wires=1)
    qml.RY(params[9], wires=2)
    qml.CNOT(wires=[1, 2])
    qml.RY(params[10], wires=5)
    qml.RY(params[11], wires=6)
    qml.CNOT(wires=[6, 5])

    # Third layer
    qml.RY(params[12], wires=2)
    qml.RY(params[13], wires=5)
    qml.CNOT(wires=[2, 5])

    # Measurement
    return qml.expval(qml.PauliZ(4))

## Circuit example

In [45]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911]

Example features: [ 7.06373117  0.09482593 -0.14447133 -3.17356484  1.51594089  1.10768911
  0.77324155 -0.67752215]

Expectation value: -0.030530278800912136

 0: ──RY(7.06)────RY(0.375)───╭C─────────────────────────────────┤     
 1: ──RY(0.0948)──RY(0.951)───╰X──RY(0.601)───╭C─────────────────┤     
 2: ──RY(-0.144)──RY(0.732)───╭C──RY(0.708)───╰X──RY(0.832)──╭C──┤     
 3: ──RY(-3.17)───RY(0.599)───╰X─────────────────────────────│───┤     
 4: ──RY(1.52)────RY(0.156)───╭X─────────────────────────────│───┤ ⟨Z⟩ 
 5: ──RY(1.11)────RY(0.156)───╰C──RY(0.0206)──╭X──RY(0.212)──╰X──┤     
 6: ──RY(0.773)───RY(0.0581)──╭X──RY(0.97)────╰C─────────────────┤     
 7: ──RY(-0.678)──RY(0.866)───╰C─────────────────────────────────┤     



# Training

In [46]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE)
STEPS = BATCH_SIZE*EPOCHS

for i in range(STEPS):
    features = X_train[i%len(X_train)]
    expected_value = y_train[i%len(X_train)]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % 200 == 0:
        print(f"epoch {10*i//200}")

epoch 0
epoch 10
epoch 20
epoch 30
epoch 40
epoch 50
epoch 60
epoch 70
epoch 80
epoch 90
epoch 100
epoch 110
epoch 120
epoch 130
epoch 140
epoch 150
epoch 160
epoch 170
epoch 180
epoch 190


In [47]:
print("Optimized rotation angles: {}".format(params))

Optimized rotation angles: [ 0.37454012  0.95071431  0.73199394  0.59865848 -1.42718851  1.28873107
  0.05808361  0.86617615  0.60111501  0.70807258  0.02058449  0.96990985
  0.83244264  0.21233911]


# Testing

In [48]:
prediction_class = np.array([])
prediction_values = np.array([])
class_errors = 0
for example, example_class in zip(X_test, y_test):
    predicted_value = circuit(example, params)
    prediction_values = np.append(prediction_values, predicted_value)

    if predicted_value <= 0:
        prediction_class = np.append(prediction_class, -1)
        if example_class > 0:
            class_errors += 1
    else:
        prediction_class = np.append(prediction_class, 1)
        if example_class <= 0:
            class_errors += 1

In [49]:
accuracy = 1 - (class_errors/len(y_test))
print(accuracy)

0.7307171853856562
